In [1]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
import networkx as nx
from parse import read_input_file, write_output_file, read_output_file
from utils import is_valid_solution, calculate_happiness, calculate_stress_for_room
from utils import convert_dictionary
import sys

## Create necessary classes and functions

Create class to handle "cities"

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

Create a fitness function

In [3]:
class Fitness: #need to add the output for our result
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
        self.result = {}
    
    def routeDistance(self): #calculate the stress level and happiness here
        for k in range(1, len(self.route)):
            D = {} #student to room
            group_list = [ [] for _ in range(k)]
            for i in range(k):
                D[self.route[i]] = i
                group_list[i].append(self.route[i])
            for i in range(k, len(self.route)):
                cur = self.route[i]
                # print(type(cur_node), cur_node)
                mini = float('inf')
                maxi = -float('inf')
                cur_group = None
                stress_group = None
                stress_out = False
                for group_num, group in enumerate(group_list):
                    happiness = sum(G[cur][other_person]['happiness'] for other_person in group if G.has_edge(cur, other_person))
                    group_copy = list(group)
                    group_copy.append(cur)
                    stress = calculate_stress_for_room(group_copy, G)
                    if stress < mini:
                        mini = stress
                        stress_group = group_num
                    if stress > s/k:
                        continue
                    if happiness > maxi:
                        cur_group = group_num
                        maxi = happiness # we can add tiebreaking here
                if cur_group == None:
                    cur_group = stress_group
                    stress_out = True
                D[cur] = cur_group
                group_list[cur_group].append(cur)
            if is_valid_solution(D, G, s, k):
                cur_val = calculate_happiness(D, G)
                self.result = D
                return cur_val
        return -1
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = float(self.routeDistance())
        return self.fitness

## Create our initial population

Route generator

In [4]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

Create first "population" (list of routes)

In [5]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

## Create the genetic algorithm

Rank individuals

In [6]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

Create a selection function that will be used to make the list of parent routes

In [7]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

Create mating pool

In [8]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

Create a crossover function for two parents to create one child

In [9]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

Create function to run crossover over full mating pool

In [10]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

Create function to mutate a single route

In [11]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

Create function to run mutation over entire population

In [12]:
def mutatePopulation(population, mutationRate):
    mutatedPop = [population[0]]
    
    for ind in range(1, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

Put all steps together to create the next generation

In [13]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

Final step: create the genetic algorithm

In [14]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial happiness: " + str(rankRoutes(pop)[0][1]))
    happiness = -float('inf')
    best = {}
    for i in range(0, generations):
        print(i)
        elite = rankRoutes(pop)[0]
        ans = Fitness(pop[elite[0]])
        if happiness < ans.routeFitness():
            best = ans.result
            happiness = elite[1]
        print("best so far:" + str(happiness))
        print(elite[1])
        print(ans.result)
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    print("Final happiness: " + str(happiness))
    print(best)
    return best

# Inputs for Our Algorithm
cityList is the student list.

In [15]:
path = "inputs/large/large-2.in" #change the input path here
G, s = read_input_file(path)
# pathout = "50.out"

# D = read_output_file(pathout, G, s)
# print("Our happiness:" + str(calculate_happiness(D, G)))
lst = list(G.nodes)
cityList = lst
random.shuffle(lst)
geneticAlgorithm(population=cityList, popSize=15, eliteSize=2, mutationRate=0.02, generations=100) #adjust hyper parameter

Initial happiness: 31.8
0
best so far:31.8
31.8
{38: 0, 16: 1, 24: 2, 31: 3, 39: 4, 32: 5, 20: 6, 23: 7, 19: 8, 17: 9, 22: 10, 47: 11, 7: 12, 49: 13, 29: 14, 8: 15, 26: 16, 11: 17, 15: 18, 41: 19, 45: 20, 5: 21, 10: 22, 0: 23, 36: 24, 13: 25, 46: 26, 42: 27, 30: 28, 37: 29, 6: 30, 12: 31, 28: 32, 4: 33, 35: 0, 27: 2, 25: 7, 18: 18, 43: 27, 40: 19, 48: 20, 9: 33, 44: 13, 1: 23, 3: 21, 34: 4, 21: 6, 14: 8, 2: 30, 33: 5}
1
best so far:40.306999999999995
40.306999999999995
{36: 0, 2: 1, 43: 2, 17: 3, 9: 4, 10: 5, 31: 6, 45: 7, 23: 8, 6: 9, 21: 10, 18: 11, 20: 12, 49: 13, 42: 14, 1: 15, 8: 16, 44: 17, 25: 18, 14: 19, 35: 20, 32: 21, 46: 22, 28: 23, 16: 24, 4: 25, 37: 26, 27: 27, 12: 28, 24: 12, 3: 1, 0: 15, 26: 10, 7: 25, 39: 26, 48: 7, 40: 17, 11: 5, 34: 6, 5: 16, 38: 20, 41: 14, 22: 8, 19: 19, 15: 11, 33: 21, 47: 17, 29: 27, 30: 26, 13: 28}
2
best so far:40.307
40.307
{18: 0, 20: 1, 49: 2, 42: 3, 1: 4, 8: 5, 44: 6, 25: 7, 14: 8, 35: 9, 32: 10, 46: 11, 28: 12, 16: 13, 4: 14, 36: 15, 2: 16,

20
best so far:45.85
45.85
{23: 0, 30: 1, 45: 2, 4: 3, 6: 4, 36: 5, 18: 6, 20: 7, 17: 8, 9: 9, 41: 10, 43: 11, 1: 12, 28: 13, 44: 14, 25: 15, 3: 16, 35: 17, 32: 18, 46: 19, 8: 20, 16: 21, 10: 22, 37: 23, 27: 24, 12: 25, 24: 26, 11: 22, 14: 8, 26: 7, 40: 10, 39: 23, 42: 11, 21: 26, 2: 16, 48: 2, 49: 14, 7: 3, 0: 12, 29: 24, 34: 23, 5: 20, 38: 17, 19: 8, 15: 6, 33: 18, 47: 14, 22: 0, 31: 1, 13: 25}
21
best so far:45.95
45.95
{35: 0, 32: 1, 20: 2, 8: 3, 16: 4, 23: 5, 45: 6, 24: 7, 6: 8, 36: 9, 18: 10, 17: 11, 43: 12, 1: 13, 44: 14, 25: 15, 9: 16, 28: 17, 14: 18, 46: 19, 3: 20, 41: 21, 30: 22, 10: 23, 37: 24, 27: 25, 12: 26, 39: 24, 42: 12, 38: 0, 19: 18, 33: 1, 40: 21, 4: 16, 11: 23, 26: 2, 47: 14, 21: 7, 2: 20, 48: 6, 49: 14, 7: 16, 0: 13, 29: 25, 34: 24, 5: 3, 15: 10, 22: 5, 31: 22, 13: 26}
22
best so far:45.95
45.95
{35: 0, 32: 1, 20: 2, 8: 3, 16: 4, 23: 5, 45: 6, 24: 7, 6: 8, 36: 9, 18: 10, 17: 11, 43: 12, 1: 13, 44: 14, 25: 15, 9: 16, 28: 17, 14: 18, 46: 19, 3: 20, 41: 21, 30: 22, 10

41
best so far:51.05000000000001
51.05000000000001
{10: 0, 20: 1, 8: 2, 43: 3, 23: 4, 45: 5, 26: 6, 35: 7, 24: 8, 6: 9, 18: 10, 17: 11, 3: 12, 41: 13, 30: 14, 16: 15, 1: 16, 44: 17, 25: 18, 36: 19, 9: 20, 37: 21, 14: 22, 46: 23, 32: 24, 15: 25, 28: 18, 49: 17, 12: 15, 39: 21, 42: 3, 38: 7, 19: 22, 33: 24, 2: 12, 4: 20, 11: 0, 22: 4, 21: 1, 40: 13, 48: 5, 27: 8, 7: 20, 0: 16, 29: 8, 34: 21, 5: 2, 47: 17, 31: 14, 13: 25}
42
best so far:51.05000000000001
51.05000000000001
{10: 0, 20: 1, 8: 2, 43: 3, 23: 4, 45: 5, 26: 6, 35: 7, 24: 8, 6: 9, 18: 10, 17: 11, 3: 12, 41: 13, 30: 14, 16: 15, 1: 16, 44: 17, 25: 18, 36: 19, 9: 20, 37: 21, 14: 22, 46: 23, 32: 24, 15: 25, 28: 18, 49: 17, 12: 15, 39: 21, 42: 3, 38: 7, 19: 22, 33: 24, 2: 12, 4: 20, 11: 0, 22: 4, 21: 1, 40: 13, 48: 5, 27: 8, 7: 20, 0: 16, 29: 8, 34: 21, 5: 2, 47: 17, 31: 14, 13: 25}
43
best so far:51.05000000000001
51.05000000000001
{10: 0, 20: 1, 8: 2, 43: 3, 23: 4, 45: 5, 26: 6, 35: 7, 24: 8, 6: 9, 18: 10, 17: 11, 3: 12, 41: 13, 30:

61
best so far:51.05000000000001
51.05000000000001
{10: 0, 20: 1, 8: 2, 43: 3, 23: 4, 45: 5, 26: 6, 35: 7, 24: 8, 6: 9, 18: 10, 17: 11, 3: 12, 41: 13, 30: 14, 16: 15, 1: 16, 44: 17, 25: 18, 36: 19, 9: 20, 37: 21, 14: 22, 46: 23, 32: 24, 15: 25, 28: 18, 49: 17, 12: 15, 39: 21, 42: 3, 38: 7, 19: 22, 33: 24, 2: 12, 4: 20, 11: 0, 22: 4, 21: 1, 40: 13, 48: 5, 27: 8, 7: 20, 0: 16, 29: 8, 34: 21, 5: 2, 47: 17, 31: 14, 13: 25}
62
best so far:51.05000000000001
51.05000000000001
{10: 0, 20: 1, 8: 2, 43: 3, 23: 4, 45: 5, 26: 6, 35: 7, 24: 8, 6: 9, 18: 10, 17: 11, 3: 12, 41: 13, 30: 14, 16: 15, 1: 16, 44: 17, 25: 18, 36: 19, 9: 20, 37: 21, 14: 22, 46: 23, 32: 24, 15: 25, 28: 18, 49: 17, 12: 15, 39: 21, 42: 3, 38: 7, 19: 22, 33: 24, 2: 12, 4: 20, 11: 0, 22: 4, 21: 1, 40: 13, 48: 5, 27: 8, 7: 20, 0: 16, 29: 8, 34: 21, 5: 2, 47: 17, 31: 14, 13: 25}
63
best so far:51.05000000000001
51.05000000000001
{10: 0, 20: 1, 8: 2, 43: 3, 23: 4, 45: 5, 26: 6, 35: 7, 24: 8, 6: 9, 18: 10, 17: 11, 3: 12, 41: 13, 30:

KeyboardInterrupt: 

## Running the genetic algorithm

Create list of cities

In [17]:
cityList = []

for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

Run the genetic algorithm

In [18]:
geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)

Initial distance: 2249.7240306428266
Final distance: 804.4841773781344


[(163,56),
 (140,97),
 (177,120),
 (187,141),
 (134,172),
 (113,170),
 (69,159),
 (46,194),
 (36,196),
 (14,184),
 (37,176),
 (45,152),
 (50,153),
 (68,116),
 (77,85),
 (29,72),
 (27,64),
 (71,50),
 (54,20),
 (59,3),
 (105,32),
 (135,38),
 (167,14),
 (192,14),
 (174,43)]

## Plot the progress

Note, this will win run a separate GA

In [ ]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

Run the function with our assumptions to see how distance has improved in each generation

In [ ]:
geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=1000)